In [1]:
using InteractiveUtils

In [2]:
ff(f, x) = f(f(x))

ff (generic function with 1 method)

In [3]:
ff(x -> 3 * x, 2)

18

In [4]:
@code_typed ff(x -> 3 * x, 2)

CodeInfo(
1 ─ %1 = Base.mul_int(3, x)::Int64
│   %2 = Base.mul_int(3, %1)::Int64
└──      return %2
) => Int64

In [5]:
function mult2_seq(xs)
    ys = ()
    for x in xs
        ys = ys..., x * 2
    end
    return ys
end

mult2_seq (generic function with 1 method)

In [6]:
mult2_seq((10, 20, 30))

(20, 40, 60)

In [7]:
map(x -> 2 * x, [1, 2, 3])

3-element Vector{Int64}:
 2
 4
 6

In [8]:
@code_lowered map(x -> 2 * x, [1, 2, 3])

CodeInfo(
1 ─ %1 = Base.Generator(f, A)
│   %2 = Base.collect_similar(A, %1)
└──      return %2
)

In [9]:
function dfa_parity(xs)
    ys = ()
    state = :s0
    for x in xs
        if state == :s0
            if x == 0
                state = :s0
                ys = ys..., :even
            elseif x == 1
                state = :s1
                ys = ys..., :odd
            else
                error("Unexpected input: ", x)
            end
        elseif state == :s1
            if x == 0
                state = :s1
                ys = ys..., :odd
            elseif x == 1
                state = :s0
                ys = ys..., :even
            else
                error("Unexpected input: ", x)
            end
        end
    end
    return ys
end

dfa_parity (generic function with 1 method)

In [10]:
dfa_parity((0, 1, 0, 1, 0))

(:even, :odd, :odd, :even, :even)

In [11]:
function dfa_run(tr, start, xs)
    ys = ()
    state = start
    for x in xs
        (s, y) = tr(state, x)
        state = s
        ys = ys..., y
    end
    return ys
end

dfa_run (generic function with 1 method)

In [12]:
function tr_parity1(state, x)
    if state == :s0
        if x == 0
            :s0, :even
        elseif x == 1
            :s1, :odd
        else
            error("Unexpected input: ", x)
        end
    elseif state == :s1
        if x == 0
            :s1, :odd
        elseif x == 1
            :s0, :even
        else
            error("Unexpected input: ", x)
        end
    end
end

tr_parity1 (generic function with 1 method)

In [13]:
dfa_run(tr_parity1, :s0, (0, 1, 0, 1, 0))

(:even, :odd, :odd, :even, :even)

In [14]:
parity_table =
    (:s0, (
        (0, (:s0, :even)),
        (1, (:s1, :odd)))),
    (:s1, (
        (0, (:s1, :odd)),
        (1, (:s0, :even))))

((:s0, ((0, (:s0, :even)), (1, (:s1, :odd)))), (:s1, ((0, (:s1, :odd)), (1, (:s0, :even)))))

In [15]:
function tr_select(tbl, state, input)
    for (s, r) in tbl
        if s == state
            for (x, t) in r
                if x == input
                    return t
                end
            end
            error("Unknown input: ", input)
        end
    end
    error("Unknown state: ", state)
end

tr_select (generic function with 1 method)

In [16]:
tr_select(parity_table, :s1, 1)

(:s0, :even)

In [17]:
function dfa_run_tbl(tbl, start, xs)
    ys = ()
    state = start
    for input in xs
        state, y = tr_select(tbl, state, input)
        ys = ys..., y
    end
    return ys
end

dfa_run_tbl (generic function with 1 method)

In [18]:
dfa_run_tbl(parity_table, :s0, (0, 1, 0, 1, 0))

(:even, :odd, :odd, :even, :even)

In [19]:
using CompTime

In [20]:
include("MacroUtils.jl")
using .MacroUtils: cleanup

In [21]:
@ct_enable function tr_select_ct(@ct(tbl), @ct(state), input)
    @ct_ctrl for (s, r) in tbl
        @ct_ctrl if s == state
            @ct_ctrl for (x, t) in r
                if @ct(x) == input
                    return @ct(t)
                end
            end
            error("Unknown input: ", input)        
        end
    end
    error("Unknown state: ", state)
end

runtime (generic function with 1 method)

In [22]:
debug(tr_select_ct, Val{parity_table}, Val{:s1}, 1) |> cleanup

quote
    if 0 == input
        return (:s1, :odd)
    end
    if 1 == input
        return (:s0, :even)
    end
    error("Unknown input: ", input)
    error("Unknown state: ", state)
end

In [23]:
tr_select_ct(Val{parity_table}, Val{:s1}, 1)

(:s0, :even)

In [24]:
# @ct_enable function dfa_run_tbl_ct(@ct(tbl), @ct(start), xs)
#     ys = ()
#     @ct state = start
#     for input in xs
#         s, y = tr_select_ct(@ct(tbl), @ct(state), input)
#         @ct state = s
#         ys = ys..., y
#     end
#     return ys
# end

In [25]:
# dfa_run_tbl_ct(Val{parity_table}, Val{:s0}, (0, 1, 0, 1, 0))

In [26]:
# debug(dfa_run_tbl_ct, Val{parity_table}, Val{:s0}, (0, 1, 0, 1, 0)) |> cleanup

In [27]:
function tr_select_state(tbl, state)
    (s, tbl_s), r... = tbl
    if s == state
        return tbl_s
    else
        tr_select_state(r, state)
    end
end

tr_select_state (generic function with 1 method)

In [28]:
function tr_select_input(tbl_s, input)
    (x, t), r... = tbl_s
    if x == input
        t
    else
        tr_select_input(r, input)
    end
end

tr_select_input (generic function with 1 method)

In [29]:
function dfa_run_tbl2(tbl, start, xs)
    ys = ()
    state = start
    for input in xs
        tbl_s = tr_select_state(tbl, state)
        state, y = tr_select_input(tbl_s, input)
        ys = ys..., y
    end
    return ys
end

dfa_run_tbl2 (generic function with 1 method)

In [30]:
dfa_run_tbl2(parity_table, :s0, (0, 1, 0, 1, 0))

(:even, :odd, :odd, :even, :even)

In [31]:
function dfa_run_tbl3(tbl, state, xs)
    if isempty(xs)
        ()
    else
        x, xs_... = xs
        tbl_s = tr_select_state(tbl, state)
        state_, y = tr_select_input(tbl_s, x)
        y, dfa_run_tbl3(tbl, state_, xs_)...
    end
end

dfa_run_tbl3 (generic function with 1 method)

In [32]:
dfa_run_tbl3(parity_table, :s0, (0, 1, 0, 1, 0))

(:even, :odd, :odd, :even, :even)

In [33]:
function tr_select_input_cont(tbl_s, input, cont)
    (x, (s, y)), r... = tbl_s
    if x == input
        cont(s , y)
    else
        tr_select_input_cont(r, input, cont)
    end
end

tr_select_input_cont (generic function with 1 method)

In [34]:
parity_table[2][2]

((0, (:s1, :odd)), (1, (:s0, :even)))

In [35]:
tr_select_input_cont(parity_table[2][2], 1, (s, y) -> (s, y))

(:s0, :even)

In [36]:
function dfa_run_cont(tbl, state, xs)
    if isempty(xs)
        ()
    else
        x, xs_... = xs
        tbl_s = tr_select_state(tbl, state)
        tr_select_input_cont(tbl_s, x,
            (s, y) -> (y, dfa_run_cont(tbl, s, xs_)...)
        )
    end
end

dfa_run_cont (generic function with 1 method)

In [37]:
dfa_run_cont(parity_table, :s0, (0, 1, 0, 1, 0))

(:even, :odd, :odd, :even, :even)

In [38]:
function tr_select_state_pe(::Val{tbl}, ::Val{state}) where {tbl,state}
    (s, tbl_s), r... = tbl
    if s == state
        Val(tbl_s)
    else
        tr_select_state_pe(Val(r), Val(state))
    end
end

tr_select_state_pe (generic function with 1 method)

In [39]:
tr_select_state_pe(Val(parity_table), Val(:s1))

Val{((0, (:s1, :odd)), (1, (:s0, :even)))}()

In [40]:
function tr_select_input_cont_pe(::Val{tbl_s}, input, cont) where {tbl_s}
    (x, (s, y)), r... = tbl_s
    if x == input
        cont(Val(s), y)
    else
        tr_select_input_cont_pe(Val(r), input, cont)
    end
end

tr_select_input_cont_pe (generic function with 1 method)

In [41]:
tr_select_input_cont_pe(Val(parity_table[2][2]), 1, (s, y) -> (s, y))

(Val{:s0}(), :even)

In [42]:
# function dfa_run_cont_pe(::Val{tbl}, ::Val{state}, xs) where {tbl, state}
#     if isempty(xs)
#         ()
#     else
#         x, xs_... = xs
#         tbl_s = tr_select_state_pe(Val(tbl), Val(state))
#         tr_select_input_cont_pe(tbl_s, x,
#             (vs, y) -> (y, dfa_run_cont_pe(Val(tbl), vs, xs_)...)
#         )
#     end
# end

In [43]:
function dfa_run_cont_pe(::Val{tbl}, ::Val{state}, xs, ys) where {tbl,state}
    if isempty(xs)
        ys
    else
        x, xs_... = xs
        tbl_s = tr_select_state_pe(Val(tbl), Val(state))
        tr_select_input_cont_pe(tbl_s, x,
            (vs, y) -> begin
                push!(ys, y)
                dfa_run_cont_pe(Val(tbl), vs, xs_, ys)
            end
        )
    end
end

dfa_run_cont_pe (generic function with 1 method)

In [44]:
# dfa_run_cont_pe(Val(parity_table), Val(:s0), [0, 1, 0, 1, 0])
dfa_run_cont_pe(Val(parity_table), Val(:s0), [0, 1, 0, 1, 0], Symbol[])

5-element Vector{Symbol}:
 :even
 :odd
 :odd
 :even
 :even

In [45]:
# @code_typed dfa_run_cont_pe(Val(parity_table), Val(:s0), [0, 1, 0, 1, 0])
@code_typed dfa_run_cont_pe(Val(parity_table), Val(:s0), [0, 1, 0, 1, 0], Symbol[])

CodeInfo(
1 ── %1  = Base.arraylen(xs)::Int64
│    %2  = (%1 === 0)::Bool
└───       goto #3 if not %2
2 ──       return ys
3 ── %5  = Base.arrayref(true, xs, 1)::Int64
│    %6  = Base.arraysize(xs, 1)::Int64
│    %7  = Base.slt_int(%6, 0)::Bool
│    %8  = Core.ifelse(%7, 0, %6)::Int64
│    %9  = Base.sle_int(2, %8)::Bool
└───       goto #5 if not %9
4 ──       goto #6
5 ──       goto #6
6 ┄─ %13 = φ (#4 => %8, #5 => 1)::Int64
│    %14 = %new(UnitRange{Int64}, 2, %13)::UnitRange{Int64}
└───       goto #7
7 ──       goto #8
8 ──       goto #13 if not true
9 ── %18 = Core.tuple(%14)::Tuple{UnitRange{Int64}}
│    %19 = Base.arraysize(xs, 1)::Int64
│    %20 = Base.slt_int(%19, 0)::Bool
│    %21 = Core.ifelse(%20, 0, %19)::Int64
│    %22 = Base.slt_int(%13, 2)::Bool
│    %23 = Base.sub_int(2, 1)::Int64
│    %24 = Base.bitcast(UInt64, %23)::UInt64
│    %25 = Base.bitcast(UInt64, %21)::UInt64
│    %26 = Base.ult_int(%24, %25)::Bool
│    %27 = Base.sub_int(%13, 1)::Int64
│    %28 = Base.bitcas